In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas.api.types import CategoricalDtype

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style()
import math as mt
from scipy import stats, linalg

In [2]:
#Caixa com todos os blocos de código
#Tratando os campos de Latitude e longitude para ajustar a base de Cidades
df = pd.read_csv('C:/Users/Asus/Downloads/Projeto_DNC/kc_house_data_tr.csv', sep = ',')
df[['lat','log']] = df[['lat','log']].astype('string')
df['lat'] = df['lat'].str.slice(stop = 6)
df['log'] = df['log'].str.slice(stop = 8)
df['chave'] = df['lat']+':'+df['log']

#Criando as tabelas unicas de Venda mais antiga e mais nova
df_max = pd.pivot_table(df, values=['data','preço'], index=['registro'], aggfunc={'data': np.max, 'preço': np.max})
df_min = pd.pivot_table(df, values=['data','preço'], index=['registro'], aggfunc={'data': np.min,'preço': np.min})
#unindo as tabelas unicas e calculando as variações de preço de venda
df_var = pd.merge(df_max, df_min, on='registro',how='left')
df_var['variacao'] = df_var['preço_x'] - df_var['preço_y']
df_var = df_var.sort_values(by='variacao',ascending=False)
df_var['%variacao'] = df_var['variacao'] / df_var['preço_y']

#fazendo o "procv" das variações de preço na base df
df = pd.merge(df, df_var, on='registro',how='left')

df

#trazendo e preparando base com nome das cidades
#df_city = pd.read_csv(r'C:\Users\matheus.cavalcante\Documents\Data_Science\DNC\Projeto_Slot3\Projeto_DNC\DNC_Deliverable_01\script\Addresses_in_King_County___address_point.csv',sep = ',',low_memory = False)
#df_city2 = df_city.get(['ZIP5','CTYNAME','LAT','LON'])
#df_city2 = df_city2.drop_duplicates()
#df_city2 = df_city2.rename(columns={'ZIP5': 'cep','LAT': 'lat','LON':'log'})
#df_city2[['lat','log']] = df_city2[['lat','log']].astype(str)
#df_city2['lat'] = df_city2['lat'].str.slice(stop = 6)
#df_city2['log'] = df_city2['log'].str.slice(stop = 8)
#df_city2['chave'] = df_city2['lat']+':'+df_city2['log']
#df_city2 = df_city2.drop_duplicates()
#df = pd.merge(df, df_city2, on='chave',how='left')
#df
#df['registro'].nunique() = 21436 -> Contagem de Casas distintas

,registro,data,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,lat,log,exclur_1,excluir_2,chave,data_x,preço_x,data_y,preço_y,variacao,%variacao
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.511,-122.257,1340,5650,47.511:-122.257,2014-10-13,221900.0,2014-10-13,221900.0,0.0,0.0
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639,47.721:-122.319,2014-12-09,538000.0,2014-12-09,538000.0,0.0,0.0
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.737,-122.233,2720,8062,47.737:-122.233,2015-02-25,180000.0,2015-02-25,180000.0,0.0,0.0
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.520,-122.393,1360,5000,47.520:-122.393,2014-12-09,604000.0,2014-12-09,604000.0,0.0,0.0
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.616,-122.045,1800,7503,47.616:-122.045,2015-02-18,510000.0,2015-02-18,510000.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.699,-122.346,1530,1509,47.699:-122.346,2014-05-21,360000.0,2014-05-21,360000.0,0.0,0.0
21609,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.510,-122.362,1830,7200,47.510:-122.362,2015-02-23,400000.0,2015-02-23,400000.0,0.0,0.0
21610,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.594,-122.299,1020,2007,47.594:-122.299,2014-06-23,402101.0,2014-06-23,402101.0,0.0,0.0
21611,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.534,-122.069,1410,1287,47.534:-122.069,2015-01-16,400000.0,2015-01-16,400000.0,0.0,0.0


In [3]:
#Vizualização resumo de quantidade de registros (vendas) por casa
df4 = df.groupby(['registro']).count()
df4.sort_values(by=['data'],ascending=False)

,data,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,lat,log,exclur_1,excluir_2,chave,data_x,preço_x,data_y,preço_y,variacao,%variacao
registro,,,,,,,,,,,,,,,,,,,,,,,,,,,
795000620,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1000102,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3578401060,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3528000040,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
4345000510,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2768000295,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2768000270,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2767900355,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [4]:
#df['cidade'] = df3.ISIN.map(df1.set_index('ISIN')['Security'])
#remove column ISIN
#df3.drop('ISIN', axis=1, inplace=True)
#df5 = pd.merge(df, df_city2, on='chave',how='left')
#df5 = df.merge(df_city2[['chave','City']],how='left',on='Person ID')
#df5 = df5.drop(columns=['exclur_1','excluir_2','excluir_3','lat_y','log_y','cep_y'], axis=1)
#df5 = df5.drop_duplicates()
#df5

In [4]:
trad = {
    'id' : 'registro',
    'date' : 'data', 
    'price' : 'preço', 
    'bedrooms' : 'quartos', 
    'bathrooms' : 'banheiros', 
    'sqft_living' : 'casa_M²',
    'sqft_lot' : 'lote_M²', 
    'floors' : 'nº_pavimentos', 
    'waterfront' : 'frente_mar', 
    'view' : 'visao', 
    'condition' : 'acabemento', 
    'grade' : 'padrao',
    'sqft_above' : 'area_casa_M²', 
    'sqft_basement' : 'subsolo_M²', 
    'yr_built' : 'construção', 
    'yr_renovated' : 'reforma', 
    'zipcode' : 'código-Postal',
    'lat' : 'latitude', 
    'long' : 'logitude', 
    'sqft_living15' : 'M²_casas_vizinhança', 
    'sqft_lot15' : 'M²_lotes_vizinhança'
    
    
}

In [2]:
df_new = df.rename(columns = {
    'id' : 'registro',
    'date' : 'data', 
    'price' : 'preço', 
    'bedrooms' : 'quartos', 
    'bathrooms' : 'banheiros', 
    'sqft_living' : 'casa_m²',
    'sqft_lot' : 'lote_m²', 
    'floors' : 'nº_pavtos', 
    'waterfront' : 'frente_mar', 
    'view' : 'visao', 
    'condition' : 'condicao', 
    'grade' : 'padrao',
    'sqft_above' : 'area_externa', 
    'sqft_basement' : 'subsolo_m²', 
    'yr_built' : 'construção', 
    'yr_renovated' : 'reforma', 
    'zipcode' : 'cep',
    'lat' : 'lat', 
    'long' : 'log', 
    'sqft_living15' : 'area_casa', 
    'sqft_lot15' : 'area_lote15'
    
    
})

NameError: name 'df' is not defined

In [ ]:
#df_new.drop(columns=['exclur_1','excluir_2','excluir_3','cep'], axis=1)

In [5]:
df.describe()

,registro,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,exclur_1,excluir_2,preço_x,preço_y,variacao,%variacao
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,2.161300e+04,2.161300e+04,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,1986.552492,12768.455652,5.412105e+05,5.389669e+05,2243.649748,0.009291
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,685.391304,27304.179631,3.667587e+05,3.674907e+05,21489.325373,0.093242
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,399.000000,651.000000,7.500000e+04,7.500000e+04,0.000000,0.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,1490.000000,5100.000000,3.240000e+05,3.200000e+05,0.000000,0.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,1840.000000,7620.000000,4.500000e+05,4.500000e+05,0.000000,0.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,2360.000000,10083.000000,6.450000e+05,6.430000e+05,0.000000,0.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,6210.000000,871200.000000,7.700000e+06,7.700000e+06,529950.000000,3.218009


In [ ]:
# PONTUAÇÃO DOS QUARTOS

In [9]:
quartos = df['quartos'].tolist()

In [10]:
qu1 = []

for x in quartos:
    if x > 7:
        qu1.append(0)
    else:
        qu1.append(x)
print(qu1[0:10])

[3, 3, 2, 4, 3, 4, 3, 3, 3, 3]


In [ ]:
# PONTUAÇÃO DOS BANHEIROS

In [107]:
banheiros = df['banheiros'].tolist()
banheiros2 = []
for x in banheiros:
    x = mt.ceil(x)
    banheiros2.append(x)
print(len(banheiros2))

21613


In [12]:
bn = []
for x in banheiros2:
    if x == 0:
        bn.append(0)
    elif x == 1:
        bn.append(1)
    elif x == 2:
        bn.append(2)
    elif x == 3:
        bn.append(3)
    elif x == 4:
        bn.append(4)
    elif x == 5:
        bn.append(5)
    elif x == 6:
        bn.append(6)
    elif x == 7:
        bn.append(7)
    elif x == 8:
        bn.append(8)
        
print(len(bn))

21613


In [ ]:
# PONTUAÇÃO DA CONDIÇÃO DA CASA

In [13]:
condicao = df['condicao'].tolist()

In [14]:
cd = []
for x in condicao:
    if x == 1:
        cd.append(1)
    elif x == 2:
        cd.append(2)
    elif x == 3:
        cd.append(3)
    elif x == 4:
        cd.append(4)
    elif x == 5:
        cd.append(5)
        
print(len(cd))

21613


In [ ]:
# PONTUAÇÃO DO PADRÃO DE QUALIDADE DA CASA

In [15]:
padrao = df['padrao'].tolist()
print(len(padrao))

21613


In [16]:
pr = []
for x in padrao:
    if x < 3:
        pr.append(1)
    elif 2 < x < 5 :
        pr.append(2)
    elif 4 < x < 7:
        pr.append(3)
    elif 6 < x < 9:
        pr.append(4)
    elif 8 < x < 11:
        pr.append(5)
    elif 10 < x <13 :
        pr.append(6)
    elif x > 12:
        pr.append(7)
        
print(len(pr))

21613


In [ ]:
# PONTUAÇÃO DA VISTA DA CASA

In [17]:
visao = df['visao'].tolist()

In [18]:
vs = []
for x in visao:
    if x == 0:
        vs.append(0)
    elif x == 1:
        vs.append(1)
    elif x == 2:
        vs.append(2)
    elif x == 3:
        vs.append(3)
    elif x == 4:
        vs.append(4)
        
print(len(vs))

21613


In [ ]:
# PONTUAÇÃO DA VISTA FRENTE AO MAR

In [19]:
frente_mar = df['frente_mar'].tolist()

In [20]:
fm = []
for x in frente_mar:
    if x == 0:
        fm.append(0)
    elif x == 1:
        fm.append(1)
        
print(len(fm))

21613


In [ ]:
# PONTUAÇÃO DO NÚMERO DE PISOS

In [21]:
nº_pavto = df['nº_pavtos'].tolist()
nº_pavtos = []
for x in nº_pavto:
    nº_pavtos.append(int(mt.ceil(x)))
print(type(nº_pavtos[0]))

<class 'int'>


In [22]:
pv = []
for x in nº_pavtos:
    if x == 1:
        pv.append(0)
    elif x == 2:
        pv.append(1)
    elif x == 3:
        pv.append(2)
    elif x == 4:
        pv.append(3)
    elif x == 5:
        pv.append(4)
    elif x == 6:
        pv.append(5)

print(len(pv))

21613


In [ ]:
# PONTUAÇÃO DE ACORCO COM A QUALIDADE DO CEP QUE A CASA ESTÁ

In [23]:
ZIPs = df['cep'].tolist()

In [24]:
cep1PONTO = [98074, 98029, 98052, 98019, 98075, 98065, 98038]
cepn1PONTO = [98057, 98050, 98224, 98134, 98070, 98288, 98148]
cepp = [] 

for x in ZIPs:
    if x in cep1PONTO:
        cepp.append(1)
    elif x in cepn1PONTO:
        cepp.append(-1)
    else:
        cepp.append(0)
        
print(len(cepp))

21613


In [ ]:
# PONTUAÇÃO QUANTO A ÁREA DA CASA

In [27]:
casa_m = df['casa_m²'].tolist()
casa_m2 = []
for x in casa_m:
    casa_m2.append(round(x/3.28,2))
print(casa_m2[0:10])
1427
1910
2550

[359.76, 783.54, 234.76, 597.56, 512.2, 1652.44, 522.87, 323.17, 542.68, 576.22]


2550

In [28]:
cm2 = []
for y in casa_m2:
  if y <= 1427:
    cm2.append(0)
  if 1427 < y <= 1910:
    cm2.append(1)
  if 1910 < y <= 2550:
    cm2.append(2)
  if y > 2550:
    cm2.append(4) 
    
print(len(cm2))


21613


In [ ]:
# PONTUAÇÃO QUANTO A ÁREA DO LOTE

In [29]:
lotem = df['lote_m²'].tolist()
lotem2 = []
for x in lotem:
    lotem2.append(round(x/3.28,2))
print(lotem2[0:10])
print(lotem[0:10])



[1722.56, 2207.93, 3048.78, 1524.39, 2463.41, 31076.22, 2078.96, 2960.67, 2277.44, 2000.0]
[5650, 7242, 10000, 5000, 8080, 101930, 6819, 9711, 7470, 6560]


In [30]:
lm2 = []
for y in casa_m2:
  if y <= 5040:
    lm2.append(0)
  if 5040 < y <= 7618:
    lm2.append(1)
  if 7618 < y <= 10688:
    lm2.append(2)
  if y > 10688:
    lm2.append(4) 
    
print(len(lm2))

21613


In [ ]:
# PONTUAÇÃO QUANTO A PRESENÇA DE ESPAÇO EM SUBSOLO

In [31]:
sbs2 = df['subsolo_m²'].tolist()
subsolo = []
for x in sbs2:
    subsolo.append(round(x/3.28,2))
print(subsolo[0:50])

sbs3 = []

for x in subsolo:
    if x != 0:
        sbs3.append(1)
    else:
        sbs3.append(0)
        
print(sbs3[0:20])
    
    


[0.0, 121.95, 0.0, 277.44, 0.0, 466.46, 0.0, 0.0, 222.56, 0.0, 518.29, 91.46, 0.0, 0.0, 0.0, 295.73, 0.0, 0.0, 0.0, 0.0, 231.71, 219.51, 0.0, 0.0, 0.0, 0.0, 213.41, 0.0, 222.56, 0.0, 0.0, 0.0, 250.0, 0.0, 237.8, 240.85, 222.56, 0.0, 100.61, 0.0, 0.0, 493.9, 0.0, 0.0, 109.76, 0.0, 0.0, 0.0, 0.0, 179.27]
['não', 'sim', 'não', 'sim', 'não', 'sim', 'não', 'não', 'sim', 'não', 'sim', 'sim', 'não', 'não', 'não', 'sim', 'não', 'não', 'não', 'não']


In [ ]:
# PONTUAÇÃO QUANTO AO ANO DE CONSTRUÇÃO

In [32]:
construcao = df['construção'].tolist()
1951, 1975, 1997

(1951, 1975, 1997)

In [33]:
const = []
for y in construcao:
  if y <= 1951:
    const.append(0)
  if 1951 < y <= 1975:
    const.append(1)
  if 1975 < y <= 1997:
    const.append(2)
  if y > 1997:
    const.append(3) 
    
print(len(const))

21613


In [ ]:
# PONTUAÇÃO AO ANO DE REFORMA

In [34]:
reforma = df['reforma'].tolist()
print(len(reforma))

21613


In [35]:
rfm = []
for x in reforma:
    if x != 0:
        rfm.append(x)
print(len(rfm))

yrfm = [] 
for x in reforma:
    if x >= 2000:
        yrfm.append(3)
    if 1980 <= x < 2000:
        yrfm.append(2)
    if 1960 <= x < 1980:
        yrfm.append(1)
    if x < 1960:
        yrfm.append(0)
        
print(len(yrfm))


914
21613


In [36]:
df['pontos_constr']

KeyError: 'pontos_constr'

In [37]:
df.drop(df[df['pontos_constr'] == 0].index, inplace=True)

KeyError: 'pontos_constr'

In [38]:
df = df.reset_index(drop=True)

In [39]:
df.head(50)

,registro,data,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,lat,log,exclur_1,excluir_2,chave,data_x,preço_x,data_y,preço_y,variacao,%variacao
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.511,-122.257,1340,5650,47.511:-122.257,2014-10-13,221900.0,2014-10-13,221900.0,0.0,0.0
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639,47.721:-122.319,2014-12-09,538000.0,2014-12-09,538000.0,0.0,0.0
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.737,-122.233,2720,8062,47.737:-122.233,2015-02-25,180000.0,2015-02-25,180000.0,0.0,0.0
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.520,-122.393,1360,5000,47.520:-122.393,2014-12-09,604000.0,2014-12-09,604000.0,0.0,0.0
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.616,-122.045,1800,7503,47.616:-122.045,2015-02-18,510000.0,2015-02-18,510000.0,0.0,0.0
5,7237550310,2014-05-12,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.656,-122.005,4760,101930,47.656:-122.005,2014-05-12,1225000.0,2014-05-12,1225000.0,0.0,0.0
6,1321400060,2014-06-27,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.309,-122.327,2238,6819,47.309:-122.327,2014-06-27,257500.0,2014-06-27,257500.0,0.0,0.0
7,2008000270,2015-01-15,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.409,-122.315,1650,9711,47.409:-122.315,2015-01-15,291850.0,2015-01-15,291850.0,0.0,0.0
8,2414600126,2015-04-15,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.512,-122.337,1780,8113,47.512:-122.337,2015-04-15,229500.0,2015-04-15,229500.0,0.0,0.0
9,3793500160,2015-03-12,323000.0,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.368,-122.031,2390,7570,47.368:-122.031,2015-03-12,323000.0,2015-03-12,323000.0,0.0,0.0


In [ ]:
# PREÇO DA CASA

In [40]:
prec = df['preço'].tolist()
preco = []
for x in prec:
    y = x*5.18
    preco.append(y)
    
print(preco[0:10])


[1149442.0, 2786840.0, 932400.0, 3128720.0, 2641800.0, 6345500.0, 1333850.0, 1511783.0, 1188810.0, 1673140.0]


In [41]:
# VALOR POR METRO QUADRADO
valor_cm2 = []
for x, y in zip(preco, casa_m2):
    valor_cm2.append(x/y)
print(valor_cm2[0:10])

[3195.0244607516124, 3556.729713862726, 3971.7157948543195, 5235.825691143986, 5157.750878563061, 3840.0789136065455, 2551.016505058619, 4677.980629390104, 2190.627994398172, 2903.6479122557357]


In [42]:
IDs = df['registro'].tolist()
print(len(IDs))

21613


In [109]:
# SOMA DE TODAS AS PONTUAÇÕES
d = []
for b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12 in zip(cm2, bn, cd, pr, vs, fm, pv, cepp, qu1, lm2,const, yrfm):
        d.append(b1 + b2+ b3+ b4+ b5+ b6+ b7+ b8+ b9+ b10+ b11+ b12)
print(len(d), '\n')

rank_casas = dict()
for x, y in zip(IDs,d):
    rank_casas[x] = y

rank_casas_T = sorted(rank_casas.items(), key=lambda x: x[1], reverse = True)
rank_casas_F = sorted(rank_casas.items(), key=lambda x: x[1], reverse = False)

print(rank_casas_T[0:10], '\n')
print(rank_casas_F[0:10])


21613 

[(1225069038, 37), (6762700020, 36), (9208900037, 36), (1924059029, 35), (2303900035, 34), (7558700030, 32), (7738500731, 31), (2424049029, 31), (1247600105, 30), (3303850390, 30)] 

[(3980300371, 3), (5702500050, 5), (8658300340, 6), (859000110, 6), (6896300380, 6), (4008400515, 6), (2023049218, 6), (745000005, 6), (3421079032, 7), (723049596, 7)]


In [50]:
df['pontuacao'] = d

In [51]:
df

,registro,data,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,lat,log,exclur_1,excluir_2,chave,data_x,preço_x,data_y,preço_y,variacao,%variacao,pontuação,pontuacao
0,7129300520,2014-10-13,1149442.00,3,1.00,359.76,1722.56,1.0,0,0,3,7,1180,0,1955,0,98178,47.511,-122.257,1340,5650,47.511:-122.257,2014-10-13,221900.0,2014-10-13,221900.0,0.0,0.0,12,12
1,6414100192,2014-12-09,2786840.00,3,2.25,783.54,2207.93,2.0,0,0,3,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639,47.721:-122.319,2014-12-09,538000.0,2014-12-09,538000.0,0.0,0.0,15,15
2,5631500400,2015-02-25,932400.00,2,1.00,234.76,3048.78,1.0,0,0,3,6,770,0,1933,0,98028,47.737,-122.233,2720,8062,47.737:-122.233,2015-02-25,180000.0,2015-02-25,180000.0,0.0,0.0,9,9
3,2487200875,2014-12-09,3128720.00,4,3.00,597.56,1524.39,1.0,0,0,5,7,1050,910,1965,0,98136,47.520,-122.393,1360,5000,47.520:-122.393,2014-12-09,604000.0,2014-12-09,604000.0,0.0,0.0,17,17
4,1954400510,2015-02-18,2641800.00,3,2.00,512.20,2463.41,1.0,0,0,3,8,1680,0,1987,0,98074,47.616,-122.045,1800,7503,47.616:-122.045,2015-02-18,510000.0,2015-02-18,510000.0,0.0,0.0,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,2014-05-21,1864800.00,3,2.50,466.46,344.82,3.0,0,0,3,8,1530,0,2009,0,98103,47.699,-122.346,1530,1509,47.699:-122.346,2014-05-21,360000.0,2014-05-21,360000.0,0.0,0.0,17,17
21609,6600060120,2015-02-23,2072000.00,4,2.50,704.27,1772.26,2.0,0,0,3,8,2310,0,2014,0,98146,47.510,-122.362,1830,7200,47.510:-122.362,2015-02-23,400000.0,2015-02-23,400000.0,0.0,0.0,17,17
21610,1523300141,2014-06-23,2082883.18,2,0.75,310.98,411.59,2.0,0,0,3,7,1020,0,2009,0,98144,47.594,-122.299,1020,2007,47.594:-122.299,2014-06-23,402101.0,2014-06-23,402101.0,0.0,0.0,14,14
21611,291310100,2015-01-16,2072000.00,3,2.50,487.80,728.05,2.0,0,0,3,8,1600,0,2004,0,98027,47.534,-122.069,1410,1287,47.534:-122.069,2015-01-16,400000.0,2015-01-16,400000.0,0.0,0.0,16,16


In [52]:
df = df.reset_index(drop=True)

In [53]:
df['casa_m²'] = casa_m2
df['preço'] = preco
df['lote_m²'] = lotem2

In [68]:
df.tail()

,registro,data,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,lat,log,exclur_1,excluir_2,chave,data_x,preço_x,data_y,preço_y,variacao,%variacao,pontuação,pontuacao
21608,263000018,2014-05-21,1864800.00,3,2.50,466.46,344.82,3.0,0,0,3,8,1530,0,2009,0,98103,47.699,-122.346,1530,1509,47.699:-122.346,2014-05-21,360000.0,2014-05-21,360000.0,0.0,0.0,17,17
21609,6600060120,2015-02-23,2072000.00,4,2.50,704.27,1772.26,2.0,0,0,3,8,2310,0,2014,0,98146,47.510,-122.362,1830,7200,47.510:-122.362,2015-02-23,400000.0,2015-02-23,400000.0,0.0,0.0,17,17
21610,1523300141,2014-06-23,2082883.18,2,0.75,310.98,411.59,2.0,0,0,3,7,1020,0,2009,0,98144,47.594,-122.299,1020,2007,47.594:-122.299,2014-06-23,402101.0,2014-06-23,402101.0,0.0,0.0,14,14
21611,291310100,2015-01-16,2072000.00,3,2.50,487.80,728.05,2.0,0,0,3,8,1600,0,2004,0,98027,47.534,-122.069,1410,1287,47.534:-122.069,2015-01-16,400000.0,2015-01-16,400000.0,0.0,0.0,16,16
21612,1523300157,2014-10-15,1683500.00,2,0.75,310.98,328.05,2.0,0,0,3,7,1020,0,2008,0,98144,47.594,-122.299,1020,1357,47.594:-122.299,2014-10-15,325000.0,2014-10-15,325000.0,0.0,0.0,14,14


In [102]:
worst_homes = df.sort_values(by=['pontuacao']).head(20)

In [103]:
best_homes = df.sort_values(by=['pontuacao']).tail(20)

In [105]:
# 5 CASAS MAIS BEM AVALIADAS PARA INVESTIR
best_homes.sort_values(by=['preço']).head(5)

,registro,data,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,lat,log,exclur_1,excluir_2,chave,data_x,preço_x,data_y,preço_y,variacao,%variacao,pontuação,pontuacao
4024,9175600025,2014-10-07,4144000.0,7,6.75,2280.49,12702.44,2.0,0,2,3,11,5080,2400,1953,0,98166,47.464,-122.368,2810,33190,47.464:-122.368,2014-10-07,800000.0,2014-10-07,800000.0,0.0,0.0,29,29
21050,2424059170,2015-02-19,4662000.0,5,6.00,2170.73,12440.85,2.0,0,4,3,12,5480,1640,2007,0,98006,47.545,-122.114,3440,36859,47.545:-122.114,2015-02-19,900000.0,2015-02-19,900000.0,0.0,0.0,30,30
21344,8835770170,2014-08-22,7707840.0,5,6.00,2097.56,85356.10,2.0,0,3,3,12,4070,2810,2007,0,98045,47.462,-121.779,4690,256803,47.462:-121.779,2014-08-22,1488000.0,2014-08-22,1488000.0,0.0,0.0,29,29
15668,4030100290,2014-10-01,8702400.0,5,3.50,1576.22,2194.21,3.0,1,4,3,11,3520,1650,1998,0,98155,47.756,-122.271,3020,12880,47.756:-122.271,2014-10-01,1680000.0,2014-10-01,1680000.0,0.0,0.0,29,29
6501,2626069030,2015-02-09,10049200.0,4,5.75,2201.22,68128.66,2.0,0,4,3,12,6220,1000,2000,0,98053,47.709,-122.013,2680,7593,47.709:-122.013,2015-02-09,1940000.0,2015-02-09,1940000.0,0.0,0.0,29,29


In [106]:
# 5 CASAS MAIS BEM AVALIADAS PARA NÃO SE INVESTIR
worst_homes.sort_values(by=['preço']).tail(5)

,registro,data,preço,quartos,banheiros,casa_m²,lote_m²,nº_pavtos,frente_mar,visao,condicao,padrao,excluir_3,subsolo_m²,construção,reforma,cep,lat,log,exclur_1,excluir_2,chave,data_x,preço_x,data_y,preço_y,variacao,%variacao,pontuação,pontuacao
12453,4022900951,2015-04-02,1579900.0,2,1.00,277.44,6928.35,1.0,0,0,1,6,910,0,1926,0,98155,47.771,-122.299,2000,14566,47.771:-122.299,2015-04-02,305000.0,2015-04-02,305000.0,0.0,0.000000,7,7
8600,2461900845,2014-09-03,1605800.0,1,1.00,173.78,1829.27,1.0,0,0,2,5,570,0,1918,0,98136,47.551,-122.385,1530,6000,47.551:-122.385,2014-09-03,310000.0,2014-09-03,310000.0,0.0,0.000000,7,7
8623,6303400395,2015-01-30,1683500.0,1,0.75,125.00,2632.93,1.0,0,0,2,4,410,0,1953,0,98146,47.507,-122.357,1190,8636,47.507:-122.357,2015-01-30,325000.0,2015-01-30,325000.0,0.0,0.000000,7,7
13629,2023049218,2015-03-16,2305100.0,2,1.00,283.54,2359.76,1.0,0,0,1,5,930,0,1932,0,98148,47.461,-122.324,1620,8584,47.461:-122.324,2015-03-16,445000.0,2014-07-16,105500.0,339500.0,3.218009,6,6
18645,7942600910,2014-12-16,2978500.0,1,1.00,399.39,2642.38,1.5,0,0,1,6,1310,0,1918,0,98122,47.605,-122.313,1130,4800,47.605:-122.313,2014-12-16,575000.0,2014-12-16,575000.0,0.0,0.000000,7,7
